In [36]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io as io
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 8.0)

import os.path as osp
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame

import torchvision
import torch

In [38]:
## 사용한 val_fold json file 경로
GT_JSON = '/opt/ml/baseline/level2-object-detection-level2-cv-19/DATAResearch/inference/serverbase_submission.json'

LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

In [69]:
image_info = coco.loadImgs(1472)[0]
image_info['file_name']

'test/1472.jpg'

In [70]:
annotation_id = coco.getAnnIds(imgIds=image_info['id'])
annotation_info_list = coco.loadAnns(annotation_id)

In [72]:
annotation_info_list

[{'image_id': 1472,
  'category_id': 0,
  'score': 0.15437564253807068,
  'area': 225213.65132496133,
  'bbox': [348.12213134765625,
   203.8895263671875,
   306.01495361328125,
   735.9563598632812],
  'iscrowd': 0,
  'id': 35419},
 {'image_id': 1472,
  'category_id': 9,
  'score': 0.1185765340924263,
  'area': 209563.41015301272,
  'bbox': [352.02703857421875,
   205.22866821289062,
   292.3814697265625,
   716.7465515136719],
  'iscrowd': 0,
  'id': 35420},
 {'image_id': 1472,
  'category_id': 1,
  'score': 0.06453073769807816,
  'area': 219378.81796425395,
  'bbox': [350.3594970703125,
   212.96823120117188,
   302.94305419921875,
   724.1585998535156],
  'iscrowd': 0,
  'id': 35421}]

In [39]:
gt = []

coco = COCO(GT_JSON)
   
'''
image id 별로 GT 값 추가 (여기서는 test.json을 사용하였기 때문에 GT가 없습니다.)
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                    float(annotation['score']),
                    float(annotation['bbox'][0]),
                    float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                    float(annotation['bbox'][1]),
                    (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=1.24s)
creating index...
index created!


In [41]:
i_list = [0]
flag = gt[0][0]

for i, v in enumerate(gt):
    if flag == v[0]:
        continue
    else:
        i_list.append(i)
        flag = v[0]

In [47]:
def intersection_over_union(boxes_preds, boxes_labels, box_format='corners'):
    '''
    Calculates intersection over union
    
    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes  (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct labels of Bounding Boxes  (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
    
    Returns:
        tensor: Intersection over union for all examples
    ''' 
    
    if box_format == 'midpoint':
        box1_x1 = boxes_preds[:,0:1] - boxes_preds[:,2:3] / 2
        box1_y1 = boxes_preds[:,1:2] - boxes_preds[:,3:4] / 2
        box1_x2 = boxes_preds[:,0:1] + boxes_preds[:,2:3] / 2
        box1_y2 = boxes_preds[:,1:2] + boxes_preds[:,3:4] / 2

        box2_x1 = boxes_labels[:,0:1] - boxes_labels[:,2:3] / 2
        box2_y1 = boxes_labels[:,1:2] - boxes_labels[:,3:4] / 2
        box2_x2 = boxes_labels[:,0:1] + boxes_labels[:,2:3] / 2
        box2_y2 = boxes_labels[:,1:2] + boxes_labels[:,3:4] / 2
    
    elif box_format == 'corners':   
        box1_x1 = boxes_preds[:,0:1]
        box1_y1 = boxes_preds[:,1:2]
        box1_x2 = boxes_preds[:,2:3]
        box1_y2 = boxes_preds[:,3:4]

        box2_x1 = boxes_labels[:,0:1]
        box2_y1 = boxes_labels[:,1:2]
        box2_x2 = boxes_labels[:,2:3]
        box2_y2 = boxes_labels[:,3:4]
        
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)
    
    # .clamp(0) is for the case when they do not intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    
    box1_area = abs((box1_x2 - box1_x1) * (box1_y1 - box1_y2))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y1 - box2_y2))
    
    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [54]:
def non_max_suppression(bboxes, iou_threshold, prob_threshold, box_format='corners'):
    # bboxes = [[1, 0.9, x1, y1, x2, y2],[],[]...]
    #        = [class_prediction, probability_score, x1, y1, x2, y2]
    
    #assert type(predictions) == list
    
    bboxes = [box for box in bboxes if box[1] > prob_threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []
    
    while bboxes:
        chosen_box = bboxes.pop(0)
        
        bboxes = [box for box in bboxes 
                  if box[0] != chosen_box[0] 
                  or intersection_over_union(torch.Tensor(chosen_box[2:]).view(1,-1),
                                      torch.Tensor(box[2:]).view(1,-1),
                                      box_format=box_format) < iou_threshold]
        
        bboxes_after_nms.append(chosen_box)
        
    return bboxes_after_nms

In [61]:
prediction_strings = []
file_names = []

for i in range(len(i_list)):
    if(i == len(i_list)-2):
        break
    file = gt[i_list[i]][0]
    bbox = [i[1:] for i in gt[i_list[i]:i_list[i+1]]]
    new_box = non_max_suppression(bbox, 0.5, 0.3)
    prediction_string = ''
    for i in new_box:
        prediction_string += str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + ' ' + str(i[3]) + ' ' + str(
                    i[4]) + ' ' + str(i[5]) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(file)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('nms.csv', index=None)
submission.head()

,PredictionString,image_id
0,7 0.998166561126709 219.02938842773438 455.630...,test/0000.jpg
1,3 0.919633686542511 340.2513122558594 757.7150...,test/0001.jpg
2,1 0.9743971824645996 289.17340087890625 997.43...,test/0002.jpg
3,9 0.98899906873703 166.99288940429688 883.6703...,test/0003.jpg
4,9 0.5510952472686768 443.4324951171875 658.492...,test/0004.jpg


In [63]:
len(i_list)

4869